In [25]:
import string
from pymorphy2 import MorphAnalyzer
from stop_words import get_stop_words
import annoy
from gensim.models import Word2Vec, FastText
import pickle
import numpy as np
import pandas as pd
from tqdm import tqdm



In [11]:
!wget https://www.dropbox.com/s/43l702z5a5i2w8j/gazeta_train.jsonl

--2023-04-16 14:05:58--  https://www.dropbox.com/s/43l702z5a5i2w8j/gazeta_train.jsonl
Resolving www.dropbox.com (www.dropbox.com)... 162.125.71.18
Connecting to www.dropbox.com (www.dropbox.com)|162.125.71.18|:443... connected.
HTTP request sent, awaiting response... 302 Found
Location: /s/raw/43l702z5a5i2w8j/gazeta_train.jsonl [following]
--2023-04-16 14:05:59--  https://www.dropbox.com/s/raw/43l702z5a5i2w8j/gazeta_train.jsonl
Reusing existing connection to www.dropbox.com:443.
HTTP request sent, awaiting response... 302 Found
Location: https://uc768bbcc4f798a146943c7efd2b.dl.dropboxusercontent.com/cd/0/inline/B6T1QVaYMbgy1TLuTYHCzjEFSkYKXMXpaitNfiEVr3nonDybeC5O0NDFKCP066s2O8Kkgmi_PUQvJHc8lWHZG_fqTSu2x4CJm6cky2U4RNK4aoYz2obk4QNBA3u7PHYorb74n8Hgl6o5m1vc-t-a_twUDHZru7_5LB_CI1wfhUtqNA/file# [following]
--2023-04-16 14:05:59--  https://uc768bbcc4f798a146943c7efd2b.dl.dropboxusercontent.com/cd/0/inline/B6T1QVaYMbgy1TLuTYHCzjEFSkYKXMXpaitNfiEVr3nonDybeC5O0NDFKCP066s2O8Kkgmi_PUQvJHc8lWHZG_fq

In [14]:
df = pd.read_json(path_or_buf='gazeta_train.jsonl', lines=True)

In [16]:
df.head(5)

,url,text,title,summary,date
0,https://www.gazeta.ru/financial/2011/11/30/385...,«По итогам 2011 года чистый отток может состав...,Прогноз не успевает за оттоком,"В 2011 году из России уйдет $80 млрд, считают ...",2011-11-30 18:33:39
1,https://www.gazeta.ru/business/2013/01/24/4939...,Российское подразделение интернет-корпорации G...,Google закончил поиск,"Юлия Соловьева, экс-директор холдинга «Профмед...",2013-01-24 18:20:09
2,https://www.gazeta.ru/social/2018/02/06/116393...,Басманный районный суд Москвы вечером 6 феврал...,«Фигуранты дела могут давить на свидетелей»,Суд арестовал на два месяца четверых экс-чинов...,2018-02-06 21:21:14
3,https://www.gazeta.ru/business/2013/06/21/5388...,Как повлияло вступление в ВТО на конкурентносп...,«С последних традиционно «отжимают» больше»,Мнения предпринимателей по поводу вступления в...,2013-06-21 17:43:50
4,https://www.gazeta.ru/culture/2014/12/27/a_636...,К третьему сезону «Голос» на Первом канале ста...,Третий «Голос» за Градского,На Первом канале завершился третий сезон шоу «...,2014-12-27 01:10:01


Провердем предобработку тектов статей

In [17]:
from nltk.tokenize import word_tokenize
from string import punctuation

import nltk
nltk.download('punkt')

from natasha import Doc, MorphVocab, Segmenter, NewsEmbedding, NewsMorphTagger

[nltk_data] Downloading package punkt to /Users/alexey/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


In [48]:
segmenter = Segmenter()
morph_vocab = MorphVocab()
emb = NewsEmbedding()
morph_tagger = NewsMorphTagger(emb)
punct = list(punctuation)

def natasha_lemmatize(text):
  doc = Doc(text)
  doc.segment(segmenter)
  doc.tag_morph(morph_tagger)
  for token in doc.tokens:
    token.lemmatize(morph_vocab)
  return [ _.lemma for _ in doc.tokens if _.lemma not in punct]

In [27]:
tqdm.pandas()

In [50]:
df['summary_lemmatize'] = df['summary'].progress_apply(lambda x: natasha_lemmatize(x))
df.head(5)

100%|█████████████████████████████████████████| 52400/52400 [18:38<00:00, 46.85it/s]


,url,text,title,summary,date,summary_lemmatize
0,https://www.gazeta.ru/financial/2011/11/30/385...,«По итогам 2011 года чистый отток может состав...,Прогноз не успевает за оттоком,"В 2011 году из России уйдет $80 млрд, считают ...",2011-11-30 18:33:39,"[в, 2011, год, из, россия, уйти, 80, млрд, счи..."
1,https://www.gazeta.ru/business/2013/01/24/4939...,Российское подразделение интернет-корпорации G...,Google закончил поиск,"Юлия Соловьева, экс-директор холдинга «Профмед...",2013-01-24 18:20:09,"[юлия, соловьев, экс-директор, холдинг, «, про..."
2,https://www.gazeta.ru/social/2018/02/06/116393...,Басманный районный суд Москвы вечером 6 феврал...,«Фигуранты дела могут давить на свидетелей»,Суд арестовал на два месяца четверых экс-чинов...,2018-02-06 21:21:14,"[суд, арестовать, на, два, месяц, четверо, экс..."
3,https://www.gazeta.ru/business/2013/06/21/5388...,Как повлияло вступление в ВТО на конкурентносп...,«С последних традиционно «отжимают» больше»,Мнения предпринимателей по поводу вступления в...,2013-06-21 17:43:50,"[мнение, предприниматель, по, повод, вступлени..."
4,https://www.gazeta.ru/culture/2014/12/27/a_636...,К третьему сезону «Голос» на Первом канале ста...,Третий «Голос» за Градского,На Первом канале завершился третий сезон шоу «...,2014-12-27 01:10:01,"[на, первый, канал, завершиться, третий, сезон..."


Создадим модели

In [53]:
VECTOR_SIZE = 300
WINDOW = 5

modelW2V = Word2Vec(sentences=df['summary_lemmatize'], vector_size=VECTOR_SIZE, 
                    window=WINDOW, min_count=1)
modelFT = FastText(sentences=df['summary_lemmatize'], vector_size=VECTOR_SIZE, min_count=1, 
                   window=WINDOW, workers=3)

Обучим модели

In [57]:
w2v_index = annoy.AnnoyIndex(VECTOR_SIZE ,'angular')
ft_index = annoy.AnnoyIndex(VECTOR_SIZE ,'angular')

index_map = {}


for i in range(len(df.summary_lemmatize)):
    n_w2v = 0
    n_ft = 0

    vector_w2v = np.zeros(VECTOR_SIZE)
    vector_ft = np.zeros(VECTOR_SIZE)
    for word in df.summary_lemmatize[i]:
        if word in modelW2V.wv:
            vector_w2v += modelW2V.wv[word]
            n_w2v += 1
        if word in modelFT.wv:
            vector_ft += modelFT.wv[word]
            n_ft += 1
    if n_w2v > 0:
        vector_w2v = vector_w2v / n_w2v
    if n_ft > 0:
        vector_ft = vector_ft / n_ft
    w2v_index.add_item(i, vector_w2v)
    ft_index.add_item(i, vector_ft)


w2v_index.build(10)
ft_index.build(10)

  0%|          | 0/52400 [00:00<?, ?it/s]

True

Проверим работоспособность модели

In [80]:
def get_response(question, index, model, index_map):
    question = natasha_lemmatize(question)
    vector = np.zeros(VECTOR_SIZE)
    norm = 0
    for word in question:
        if word in model.wv:
            vector += model.wv[word]
            norm += 1
    if norm > 0:
        vector = vector / norm
    answers = index.get_nns_by_vector(vector, 10, )
    return [index_map[i] for i in answers]

In [78]:
TEXT = "Московская биржа приостановила торги"

In [81]:
get_response(TEXT, w2v_index, modelW2V, df.title)

['Банк в три хода',
 'Газпромбанк ушел с биржи',
 'Dufry встала у воздушных ворот',
 'Московскую биржу разложили по пакетам',
 'Офисы выходят в «окно»',
 'Россия заходит в Канаду через Лондон',
 '«Яндекс» нашел себе цену',
 'Триста миллионов долларов за один день',
 'Основатели «Яндекса» запускают поиск денег ',
 'Инвесторы встали на якорь']